In [25]:
from glob import glob
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd

import torch

from matplotlib import pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertModel

## e060 の valid を確認する

In [26]:
trn_df = pd.read_csv('../inputs/origin/train.csv')
display(trn_df.shape, trn_df.head())

(27481, 4)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [27]:
trn_df = trn_df[trn_df.selected_text.notnull()]
trn_df.shape

(27480, 4)

In [28]:
ckpts = []
for fckpt in glob('../checkpoints/e060/best/*'):
    ckpts.append(torch.load(fckpt))

In [29]:
ckpts[0].keys()

dict_keys(['fold_num', 'current_epoch', 'model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'val_textIDs', 'val_input_ids', 'val_preds', 'val_labels', 'histories'])

In [30]:
def _get_predicted_texts(input_ids, y_preds_head, y_preds_tail, tokenizer):
    predicted_texts = []
    for input_id, y_pred_head, y_pred_tail \
            in zip(input_ids, y_preds_head, y_preds_tail):
        pred_label_head = y_pred_head.argmax()
        pred_label_tail = y_pred_tail.argmax()
        predicted_text = tokenizer.decode(
            input_id[pred_label_head:pred_label_tail])
        predicted_texts.append(predicted_text)

    return predicted_texts

In [31]:
def _get_selected_texts(input_ids, labels_head, labels_tail, tokenizer):
    predicted_texts = []
    for input_id, pred_label_head, pred_label_tail \
            in zip(input_ids, labels_head, labels_tail):
        predicted_text = tokenizer.decode(
            input_id[pred_label_head:pred_label_tail])
        predicted_texts.append(predicted_text)

    return predicted_texts

In [32]:
import sys
sys.path.append('../')
from tools.tokenizers import myRobertaByteLevelBPETokenizer

tokenizer = myRobertaByteLevelBPETokenizer(   
    vocab_file='../inputs/datasets/roberta/tokenizer/vocab.json',
    merges_file='../inputs/datasets/roberta/tokenizer/merges.txt',
    lowercase=True,
    add_prefix_space=True)

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [33]:
for ckpt in ckpts:
    trn_df = trn_df.set_index('textID')
    predicted_texts = _get_predicted_texts(ckpt['val_input_ids'], ckpt['val_preds'][0],  ckpt['val_preds'][1], tokenizer)
    trn_df.loc[ckpt['val_textIDs'], 'predicted_texts'] = predicted_texts
    trn_df = trn_df.reset_index()
trn_df

,textID,text,selected_text,sentiment,predicted_texts
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i`d have responded, if i were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sad
2,088c60f138,my boss is bullying me...,bullying me,negative,bullying me...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of ****,"
...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,lost
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,i`ve wondered
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,but it was worth it ****.


In [34]:
for ckpt in ckpts:
    trn_df = trn_df.set_index('textID')
    manual_selected_texts = _get_selected_texts(ckpt['val_input_ids'], ckpt['val_labels'][0],  ckpt['val_labels'][1], tokenizer)
    trn_df.loc[ckpt['val_textIDs'], 'manual_selected_text'] = manual_selected_texts
    trn_df = trn_df.reset_index()
trn_df

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i`d have responded, if i were going","i`d have responded, if i were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sad,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,bullying me...,bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of ****,","sons of ****,"
...,...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,lost,wish we could come see u on denver husband lo...
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,i`ve wondered,i`ve wondered about rake to. the client has m...
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good,yay good for both of you.
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,but it was worth it ****.,but it was worth it ****.


In [35]:
trn_df.query('textID == "6cd1ba8680"')

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text
19561,6cd1ba8680,Thats charmingly funny,ats charmingly fun,positive,thats charmingly funny,thats charmingly funny


In [36]:
trn_df[trn_df.manual_selected_text.str.contains('</s>')]

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text


In [37]:
trn_df['selected_text_lower'] = trn_df['selected_text'].apply(lambda x: " " + " ".join(x.lower().split()))
trn_df

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text,selected_text_lower
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"i`d have responded, if i were going","i`d have responded, if i were going","i`d have responded, if i were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,sad,sooo sad,sooo sad
2,088c60f138,my boss is bullying me...,bullying me,negative,bullying me...,bullying me,bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone,leave me alone,leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"sons of ****,","sons of ****,","sons of ****,"
...,...,...,...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,lost,wish we could come see u on denver husband lo...,d lost
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,i`ve wondered,i`ve wondered about rake to. the client has m...,", don`t force"
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,yay good,yay good for both of you.,yay good for both of you.
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,but it was worth it ****.,but it was worth it ****.,but it was worth it ****.


In [38]:
(trn_df.selected_text_lower == trn_df.manual_selected_text).mean()

0.9031659388646288

In [39]:
trn_df[(trn_df.selected_text_lower != trn_df.manual_selected_text)].query('sentiment != "neutral"').head(100)

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text,selected_text_lower
18,af3fed7fc3,is back home now gonna miss every one,onna,negative,miss every one,is back home now gonna miss every one,onna
27,bdc32ea43c,On the way to Malaysia...no internet access to...,.no internet,negative,on the way to malaysia...no internet access t...,on the way to malaysia...no internet access t...,.no internet
32,1c31703aef,If it is any consolation I got my BMI tested ...,well so much for being unhappy for about 10 mi...,negative,unhappy for about 10 minutes.,well so much for being unhappy for about 10 m...,well so much for being unhappy for about 10 m...
39,2863f435bd,A little happy for the wine jeje ok it`sm my f...,A little happy fo,positive,happy for the wine jeje ok it`sm my free time...,a little happy for,a little happy fo
48,3d9d4b0b55,"i donbt like to peel prawns, i also dont like ...",dont like go,negative,i donbt like,dont like going,dont like go
...,...,...,...,...,...,...,...
1061,7048b3a9c0,"Nako! Umuulan pa naman! Anyway, enjoy the bi...",", enjoy",positive,enjoy the bike rides!,"nako! umuulan pa naman! anyway, enjoy the bik...",", enjoy"
1075,7a3840a246,Don`t follow your dreams; chase them.- Richard...,who I think is very smart,positive,don`t follow your dreams; chase them.- richar...,don`t follow your dreams; chase them.- richar...,who i think is very smart
1076,3d5c1ed21b,Up is out? I didn`t get the memo It looks ...,o It looks amazi,positive,amazing.,up is out? i didn`t get the memo it looks ama...,o it looks amazi
1082,e9c337f756,@_TWEE haha thanks to you that`s my new word f...,thanks to you th,positive,thanks,thanks to you that,thanks to you th


In [40]:
trn_df.shape, trn_df[(trn_df.selected_text_lower != trn_df.manual_selected_text)].shape

((27480, 7), (2661, 7))

In [41]:
trn_df['manual_and_selected_intersection_len'] = trn_df.apply(lambda row: len(set(row['manual_selected_text'].split()) & set(row['selected_text_lower'].split())), axis=1)
trn_df['manual_and_selected_intersection_len'].value_counts().sort_index()

0      555
1     7163
2     2252
3     2145
4     1971
5     1576
6     1374
7     1147
8     1007
9      879
10     805
11     737
12     659
13     586
14     553
15     506
16     466
17     453
18     440
19     393
20     413
21     334
22     313
23     290
24     212
25     114
26      75
27      40
28      14
29       8
Name: manual_and_selected_intersection_len, dtype: int64

In [42]:
trn_df['manual_and_selected_tokenized_intersection_len'] = trn_df.apply(lambda row: len(set(tokenizer.encode(' '.join(row['manual_selected_text'].split()), add_special_tokens=False).ids) & set(tokenizer.encode(' '.join(row['selected_text_lower'].split()), add_special_tokens=False).ids)), axis=1)
trn_df['manual_and_selected_tokenized_intersection_len'].value_counts().sort_index()

0      168
1     5196
2     2792
3     1845
4     1739
5     1617
6     1315
7     1148
8     1013
9      956
10     798
11     695
12     689
13     642
14     581
15     502
16     513
17     477
18     446
19     452
20     383
21     358
22     375
23     353
24     366
25     319
26     307
27     280
28     238
29     239
30     163
31     130
32     134
33      89
34      54
35      29
36      26
37      15
38      14
39       8
40       9
41       4
42       3
Name: manual_and_selected_tokenized_intersection_len, dtype: int64

In [43]:
trn_df.query('manual_and_selected_tokenized_intersection_len == 0')

,textID,text,selected_text,sentiment,predicted_texts,manual_selected_text,selected_text_lower,manual_and_selected_intersection_len,manual_and_selected_tokenized_intersection_len
18,af3fed7fc3,is back home now gonna miss every one,onna,negative,miss every one,is back home now gonna miss every one,onna,0,0
168,e56eeb4968,Few Bevvies 2day in twn..great on a day off!!,great,positive,great on a day off!!,few bevvies 2day in twn..great on a day off!!,great,0,0
189,a0a306868a,lost my tooth 2day whilst i was eating gum...oww,oww,negative,lost,lost my tooth 2day whilst i was eating gum...oww,oww,0,0
309,a54d3c2825,"I know It was worth a shot, though!",as wort,positive,"i know it was worth a shot, though!","i know it was worth a shot, though!",as wort,0,0
348,322b61740c,"90 degrees, gross skies, and thunderstorms...p...",perfect ma,positive,"gross skies, and thunderstorms...perfect matc...","90 degrees, gross skies, and thunderstorms......",perfect ma,0,0
...,...,...,...,...,...,...,...,...,...
27120,a11ddb9e58,The geographY was an exam today!But turned out...,ry nervou,negative,nervous...,the geography was an exam today!but turned ou...,ry nervou,0,0
27208,30a1e8c2b4,Guys ( ) I know! My ability to read time te...,es faile,negative,failed!,guys ( ) i know! my ability to read time tell...,es faile,0,0
27375,13fbc75291,Going to church in the morning...happy mommas ...,happy,positive,happy,going to church in the morning...happy mommas...,happy,0,0
27385,e149ebd3a1,#vwll2009 Would one of the VWLLers want to add...,ch appreciat,positive,appreciate,#vwll2009 would one of the vwllers want to ad...,ch appreciat,0,0


In [44]:
trn_df.to_csv('../inputs/nes_info/v2_dataset_trn_df.csv', index=False)

In [90]:
trn_df.loc[955]['text']

' (cont) a bastardized version of French? It`s fun, believe me!'

In [86]:
trn_df.loc[955]['selected_text_lower']

' s fun,'

In [88]:
trn_df.loc[955]['manual_selected_text']

's fun,'

In [81]:
tokenizer.encode('i also dont like going shopping').tokens

['Ġi', 'Ġalso', 'Ġdont', 'Ġlike', 'Ġgoing', 'Ġshopping']

In [65]:
trn_df.loc[27478]

textID                                   ed167662a5
text                     But it was worth it  ****.
selected_text            But it was worth it  ****.
sentiment                                  positive
predicted_texts           but it was worth it ****.
manual_selected_text      but it was worth it ****.
selected_text_lower      but it was worth it  ****.
Name: 27478, dtype: object

In [66]:
trn_df.loc[27478]['manual_selected_text']

' but it was worth it ****.'

In [68]:
trn_df.loc[27478]['selected_text_lower']

' but it was worth it  ****.'

In [69]:
trn_df.loc[27478]['manual_selected_text'] == trn_df.loc[27478]['selected_text_lower']

False

In [70]:
for i, j in zip(trn_df.loc[27478]['manual_selected_text'], trn_df.loc[27478]['selected_text_lower']):
    print(i, j)

   
b b
u u
t t
   
i i
t t
   
w w
a a
s s
   
w w
o o
r r
t t
h h
   
i i
t t
   
*  
* *
* *
* *
. *


## debug

In [27]:
row = trn_df.query('textID == "6cd1ba8680"').iloc[0]

In [36]:
text = " " + " ".join(row['text'].split())
text_output = tokenizer.encode_plus(
    text=text,
    # text_pair=None,
    # text_pair=f"[{row['sentiment']}]",
    text_pair=row['sentiment'],
    add_special_tokens=True,
    max_length=125,
    pad_to_max_length=True,
    return_tensor='pt',
    return_token_type_ids=False,
    return_attention_mask=True,
    return_special_tokens_mask=True,
)
row['input_ids'] = text_output['input_ids']
row['attention_mask'] = text_output['attention_mask']
row['special_tokens_mask'] = text_output['special_tokens_mask']

if 'selected_text' not in row:
    row['selected_text'] = ''
    row['labels_head'] = -1
    row['labels_tail'] = -1
#    return row

text = " " + " ".join(row['selected_text'].split())
selected_text_output = tokenizer.encode_plus(
    text=text,
    text_pair=None,
    add_special_tokens=False,
    max_length=125,
    pad_to_max_length=False,
    return_tensor='pt',
    return_token_type_ids=False,
    return_attention_mask=False,
)

# allign labels for segmentation
input_ids = text_output['input_ids']
sel_input_ids = selected_text_output['input_ids']
# 1 start なのは、先頭の token をスルーするため
matched_cnt = len([i for i in input_ids[1:1 + len(sel_input_ids)]
                   if i in sel_input_ids])
best_matched_cnt = matched_cnt
best_matched_i = 1
# for i in range(0, len(input_ids)):
# 1 start なのは、先頭の token をスルーするため
for i in range(1, len(input_ids) - len(sel_input_ids)):
    head_input_id_i = input_ids[i]
    tail_input_id_i = input_ids[i + len(sel_input_ids)]
    if head_input_id_i in sel_input_ids:
        matched_cnt -= 1
    if tail_input_id_i in sel_input_ids:
        matched_cnt += 1
    if matched_cnt < 0:
        raise Exception('invalid logic')

    if best_matched_cnt < matched_cnt:
        best_matched_cnt = matched_cnt
        best_matched_i = i + 1   # 抜いた時の話なので
    if best_matched_cnt == len(sel_input_ids):
        break

row['labels_head'] = best_matched_i
row['labels_tail'] = best_matched_i + len(sel_input_ids)

In [44]:
row['text']

'  Thats charmingly funny'

In [45]:
row['selected_text']

'ats charmingly fun'

In [39]:
selected_text_output['input_ids']

[23, 29, 18452, 352, 1531]

In [40]:
" " + " ".join(row['selected_text'].split())

' ats charmingly fun'

In [42]:
tokenizer.encode(" " + " ".join(row['selected_text'].split())).ids

[23, 29, 18452, 352, 1531]

In [43]:
tokenizer.encode(" " + " ".join(row['text'].split())).ids

[45365, 18452, 352, 6269]

In [ ]:
tokenizer.decode()

## tokenizer 確認

In [17]:
import sys
sys.path.append('../')
from tools.tokenizers import myRobertaByteLevelBPETokenizer

my_tokenizer = myRobertaByteLevelBPETokenizer(   
    vocab_file='../inputs/datasets/roberta/tokenizer/vocab.json',
    merges_file='../inputs/datasets/roberta/tokenizer/merges.txt',
    lowercase=True,
    add_prefix_space=True)

In [21]:
my_tokenizer.decode(ckpt['val_input_ids'][0][1:11])

' i`d have responded, if i were going'

In [52]:
def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [53]:
temp_jac = 0
for i, row in trn_df.iterrows():
    temp_jac += jaccard(row['selected_text'], row['predicted_texts'])

In [54]:
temp_jac / len(trn_df)

0.44581498228808364

## 改善の必要あり
 - decode すると違う形式になってしまう
 - selected_text を encode -> decode して形式が変わるパターンを除去する後処理が必要そう

In [75]:
trn_df.loc[0]

textID                                                         cb774db0d1
text                                  I`d have responded, if I were going
selected_text                         I`d have responded, if I were going
sentiment                                                         neutral
predicted_texts         i ` d have responded, if i were going [SEP] [PAD]
manual_selected_text    i ` d have responded, if i were going [SEP] [PAD]
Name: 0, dtype: object

In [74]:
jaccard(trn_df.loc[0]['selected_text'], trn_df.loc[0]['predicted_texts'][:-12])

0.6666666666666666

In [72]:
trn_df.loc[0]['selected_text'][4:], trn_df.loc[0]['predicted_texts'][6:-12]

('have responded, if I were going', 'have responded, if i were going')